In [87]:
import numpy as np
from numpy import nan

from tabulate import tabulate
from itertools import zip_longest 

### Environnement :
- Initialisation de 2 stack
- Affichage de 2 Stack 
    - (print du top aux bot -> Ok)
    - Afficher toutes les valeurs si l'autre ne possede pas la meme taille -> KO
    - afficher les valeurs NULL -> KO (le "-" est pourris)
- push A fonctionnent mais trouver une facons de renplir la stack qui pet un element



In [172]:
class Env():
    
    def __init__(self, size):
        self.stack_A = np.arange(size)
        self.stack_B = np.array([])
        self.stack_A_top = self.stack_A[-1]
        self.stack_B_top = None

        
    def print_stack(self):
        print("-------------------")
        for a, b in zip_longest(reversed(self.stack_A), reversed(self.stack_B), fillvalue='xxx'):
          
          print(f"{a}\t{b}")
        print("-------------------")

    def push_a(self):
        # Si A est vide 
        if not self.stack_A.size:
          return
        # Si B est vide
        if not self.stack_B.size :
          self.stack_B = np.append(self.stack_B, self.stack_A_top)
          self.stack_A = np.delete(self.stack_A, self.stack_A_top)
          self.stack_B_top = self.stack_A_top
        # SI B contient quelque chose
        else : 
          self.stack_B = np.append(self.stack_B, self.stack_A_top)
          self.stack_A = np.delete(self.stack_A, self.stack_A_top)
          self.stack_B_top = self.stack_A_top
  
        # Si A est vide maintenant
        if not self.stack_A.size:
          self.stack_A_top = None
        else:
          self.stack_A_top = self.stack_A[-1]
    
    def push_b(self):
        # Si B est vide 
        if not self.stack_B.size:
          return
        # Si A est vide
        if not self.stack_A.size :
          self.stack_A = np.append(self.stack_A, self.stack_B_top)
          self.stack_B = np.delete(self.stack_B, self.stack_B_top)
          self.stack_A_top = self.stack_B_top
        # SI A contient quelque chose
        else : 
          self.stack_A = np.append(self.stack_A, self.stack_B_top)
          print(f"B top -> {self.stack_B_top}")
          self.stack_B = np.delete(self.stack_B, self.stack_B_top)
          self.stack_A_top = self.stack_B_top
        
        # Si B est vide maintenant
        if not self.stack_B.size:
          self.stack_B_top = None
        else:
          self.stack_B_top = self.stack_B[-1]


In [173]:
test = Env(10)

In [174]:
test.print_stack()
test.push_a()
test.push_a()
test.push_a()
test.print_stack()
test.push_b()
test.push_b()
test.push_b()
test.print_stack()

-------------------
9	xxx
8	xxx
7	xxx
6	xxx
5	xxx
4	xxx
3	xxx
2	xxx
1	xxx
0	xxx
-------------------
-------------------
6	7.0
5	8.0
4	9.0
3	xxx
2	xxx
1	xxx
0	xxx
-------------------
B top -> 7


IndexError: ignored

In [154]:
a = np.arange(10)
b = np.arange(5)
for i, y in itertools.izip_longest(reversed(a), reversed(b)):
  print(i, y)


AttributeError: ignored

In [89]:
l1=['Cat', 'Dog', 'Gorilla', 'Ladybug']
l2=['Cat', 'Dog']

for a, b in zip_longest(l1, l2, fillvalue='xx'):
    print("{0:20s}\t{1:20s}".format(a, b))


Cat                 	Cat                 
Dog                 	Dog                 
Gorilla             	xx                  
Ladybug             	xx                  
